In [ ]:
pip install openai

In [1]:
# Setup
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI

# Using User Managed Identity client id
# https://learn.microsoft.com/en-us/python/api/overview/azure/identity-readme?view=azure-python#managed-identity-support
client_id = "<client_id>"

# Azure Open AI endpoint URL
endpoint_url = "<endpoint_url>"

# Azure Open AI model deployment name
model_deployment_name = "gpt-4o-mini"

#### Set of sample tickets

In [33]:
support_ticket_1 = """Subject: Unable to Access Work Email Account
Description:
Hi IT Team,
I’ve been trying to log into my work email account since this morning and I keep getting a “password incorrect” error. I’m 100% sure I’m using the correct password. I even reset it once, and the new password doesn’t work either. Could you please check if something is wrong with my account?
""".strip()

support_ticket_2 = """Subject: Error Message in Project Management Application
Description:
Hello Support,
Every time I open the Project Management desktop app, I get an error that says “Unable to connect to server.” I’ve tried restarting my computer, and my internet connection is fine. I also tested another coworker’s laptop using my login, and it worked there. It might be specific to my machine.""".strip()

support_ticket_3 = """Subject: Frequent VPN Disconnections
Description:
Hello,
My VPN connection drops randomly multiple times a day. It reconnects automatically, but it disrupts my remote desktop sessions. Is there an update or a known issue with the VPN? Could you look into possible network stability problems?"""

support_ticket_4 = """Subject: Magical Story 
Description: A long, long time ago a little boy was walking through a park. In the middle of the park there was a tree with a sign on it. The sign said "I am a magic tree. Say the magic words and you will see."
The boy tried to guess the magic words. He tried abracadabra, supercalifragilisticexpialidocious, ta-ta-ra, and many more... but none of them worked."""

support_ticket_5 = """Subject: Het werkt allemaal niet....
Description: Het papier loopt weer vast. We kunnen zo echt ons werkt niet doen. Kom onmiddelijk helpen!!!!""" 

#### Define system prompt

In [34]:
system_prompt = '''
You are an Expert Technical Support Operator classifying support tickets. These tickets can be in English or Dutch.
Give me an output using the following JSON format with four fields:
{
  "category": "CATEGORY",
  "ticket_priority": "TICKETPRIORITY",
  "sentiment": "SENTIMENT",
  "summary": "SUMMARY"
}
Return one CATEGORY from the list below based on the provided subject and description text.
If there is no matching CATEGORY, classify it as Unknown.
CATEGORY:
- Software Issues
- Account Issues
- Hardware Issues
- Connectivity Issues
- Security Issues
- Data Issues
Return one SENTIMENT that fits the provided subject and description text. 
If there is no matching SENTIMENT, classify it as Unknown.
SENTIMENT: 
- Angry
- Frustrated
- Neutral
- Satisfied
- Unknown
Return one TICKETPRIORITY that fits the provided subject and description text.
If there is no matching TICKETPRIORITY, classify it as Unknown.
TICKETPRIORITY:
- Low
- Medium
- High
- Critical
- Unknown"
SUMMARY:
Provide a short SUMMARY of 1 line maximum. 
'''

In [35]:
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(managed_identity_client_id=client_id), "https://cognitiveservices.azure.com/.default"
)

client = AzureOpenAI(
    api_version="2024-08-01-preview",
    azure_endpoint=endpoint_url,
    azure_ad_token_provider=token_provider
)

def classify_ticket(ticket_text: str):
    response = client.chat.completions.create(
        model=model_deployment_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": ticket_text},
        ]
    )
    return response

In [19]:
result1 = classify_ticket(support_ticket_1)
result2 = classify_ticket(support_ticket_2)
result3 = classify_ticket(support_ticket_3)
result4 = classify_ticket(support_ticket_4)
result5 = classify_ticket(support_ticket_5)

print(result1.choices[0].message.content)
print(result2.choices[0].message.content)
print(result3.choices[0].message.content)
print(result4.choices[0].message.content)
print(result5.choices[0].message.content)

{
  "category": "Account Issues",
  "ticket_priority": "High",
  "sentiment": "Frustrated",
  "summary": "User unable to access work email account despite correct password."
}
{
  "category": "Connectivity Issues",
  "ticket_priority": "High",
  "sentiment": "Frustrated",
  "summary": "User experiences connection error in Project Management app."
}
{
  "category": "Connectivity Issues",
  "ticket_priority": "Medium",
  "sentiment": "Frustrated",
  "summary": "User experiences frequent VPN disconnections affecting remote desktop sessions."
}
{
  "category": "Unknown",
  "ticket_priority": "Unknown",
  "sentiment": "Neutral",
  "summary": "A story about a boy and a magic tree."
}
{
  "category": "Hardware Issues",
  "ticket_priority": "Critical",
  "sentiment": "Angry",
  "summary": "Papier vastloopprobleem leidt tot ernstige hinder voor het werk."
}
